# Notebook 3 : Deep learning with Tensorflow

In this notebook we want to explore deep learning techniques if this dataset

In [1]:

#Data Loading and Exploration

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
df = pd.read_csv("Stress.csv")

not_used_cols = ['subreddit','post_id','sentence_range','confidence','social_timestamp']
df1 = df.drop(not_used_cols,axis=1)
df1['label'].unique()
#Text processing
#importing required libraries (nltk,spacy,urllib,re etc.)
#downloading required database (omw), wordnet, punkt, stopwords
#cleaning data (with regualr expression)
import nltk
import re
from urllib.parse import urlparse
#from spacy import load
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('omw-1.4') # Open Multilingual Wordnet, this is an lexical database
nltk.download('wordnet')
nltk.download('wordnet2022')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
lemmatizer = WordNetLemmatizer()
stop_words = list(stopwords.words('english'))
def textProcess(sent):
    try:
        # Replace square brackets, parentheses with spaces
        sent = re.sub('[][)(]', ' ', sent)

        # Remove URLs by checking if a word has a URL scheme
        sent = [word for word in sent.split() if not urlparse(word).scheme]
        sent = ' '.join(sent)

        # Remove escape characters starting with '@'
        sent = re.sub(r'\@\w+', '', sent)

        # Remove HTML tags using regular expressions
        sent = re.sub(re.compile("<.*?>"), '', sent)

        # Keep only letters and numbers, replace others with spaces
        sent = re.sub("[^A-Za-z0-9]", ' ', sent)

        # Convert all words to lowercase
        sent = sent.lower()

        # Strip extra spaces from words and sentences
        sent = [word.strip() for word in sent.split()]
        sent = ' '.join(sent)

        # Tokenize the text into words
        tokens = word_tokenize(sent)

        # Remove stopwords (common words that don't carry much meaning)
        tokens = [word for word in tokens if word not in stop_words]

        # Lemmatize the words (convert words to their base form)
        sent = [lemmatizer.lemmatize(word) for word in tokens]
        sent = ' '.join(sent)

        return sent

    except Exception as ex:
        # Handle exceptions and print an error message
        print(sent, "\n")
        print("Error ", ex)


In [3]:
df1['processed_text'] = df1['text'].apply(lambda text: textProcess(text))

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

**The goal now is to find the best deep learning techniques to determind stress**

Step 2: Train-Test Split and Tokenization

  The dataset is tokenized and padded for input into the LSTM model and split into training and testing sets.
  The model architecture is defined with an embedding layer, LSTM layer, and a dense output layer to visualize best option to have best score

In [ ]:
# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df1['processed_text'])
sequences = tokenizer.texts_to_sequences(df1['processed_text'])
padded_sequences = pad_sequences(sequences)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, df1['label'], test_size=0.2, stratify=df1['label'], random_state=0)


Tokenization serves as a crucial step in transforming raw text into a format suitable for deep learning models.
The creation of numerical representations essential for training models like LSTM.
Now the model can understand and learn patterns within stress-related text
Lets continue...

In [ ]:
embedding_dim = 50  # Adjust based on your choice
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=padded_sequences.shape[1]))
model.add(LSTM(units=50, activation='relu'))
model.add(Dense(units=3, activation='softmax'))  # Assuming 3 stress levels

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)


In [ ]:
# Evaluate on test set
y_pred = np.argmax(model.predict(X_test), axis=1)

# Print metrics
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df1['processed_text'], df1['label'], test_size=0.2, random_state=42)

# Tokenize the text
max_words = 5000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure consistent length for input
max_sequence_length = 100  # You can adjust this based on your data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Define the number of LSTM units
lstm_units = 64

# Create a Sequential model
model_lstm = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(LSTM(lstm_units, return_sequences=True)),
    tf.keras.layers.Bidirectional(LSTM(lstm_units)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile the model
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compile the model
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
model_lstm.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test))

# Evaluate the model
test_loss_lstm, test_accuracy_lstm = model_lstm.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (LSTM): {test_accuracy_lstm * 100:.2f}%')


Let's try using a different deep learning technique, such as a 1D Convolutional Neural Network (1D CNN).

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Define the number of filters for the Conv1D layer
num_filters = 64

# Create a Sequential model with 1D CNN
model_cnn = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile the model
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_cnn.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test))

# Evaluate the model
test_loss_cnn, test_accuracy_cnn = model_cnn.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (1D CNN): {test_accuracy_cnn * 100:.2f}%')


In [ ]:
# Compile the model
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model_cnn.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test))

# Evaluate the model
test_loss_cnn, test_accuracy_cnn = model_cnn.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (1D CNN): {test_accuracy_cnn * 100:.2f}%')


In [ ]:
# Tokenize the text
max_words = 5000
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)


In [ ]:

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


In [ ]:
# Pad sequences to ensure consistent length for input
max_sequence_length = 100  # You can adjust this based on your data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')


LSTM, used for handling sequential data, excels in managing long-range dependencies.
It incorporates memory cells and gates, such as input, output, and forget gates, to regulate the flow of information through the network.

In [ ]:
# Define the number of LSTM units
lstm_units = 64


In [ ]:
# Create a Sequential model
model_lstm = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    tf.keras.layers.Bidirectional(LSTM(lstm_units, return_sequences=True)),
    tf.keras.layers.Bidirectional(LSTM(lstm_units)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])


In [ ]:
# Compile the model
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model
epochs = 10
batch_size = 32
model_lstm.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test))


In [ ]:
# Evaluate the model
test_loss_lstm, test_accuracy_lstm = model_lstm.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (LSTM): {test_accuracy_lstm * 100:.2f}%')


CNN

CNNs have found adaptation in sequential tasks. They utilize convolutional layers to capture spatial patterns, making them effective for identifying local features within sequences.

In [ ]:
# Create a Sequential model with 1D CNN
model_cnn = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Conv1D(filters=num_filters, kernel_size=3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])


In [ ]:
# Compile the model
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model_cnn.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test))

# Evaluate the model
test_loss_cnn, test_accuracy_cnn = model_cnn.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (1D CNN): {test_accuracy_cnn * 100:.2f}%')


GRU

GRU is a variant of LSTM designed for sequential data processing with a focus on reducing computational complexity. It employs a simplified gating mechanism to efficiently handle information flow through the network.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

# Define the number of GRU units
gru_units = 64

# Create a Sequential model with GRU
model_gru = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    GRU(gru_units, return_sequences=True),
    GRU(gru_units),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])



In [ ]:
# Compile the model
model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
model_gru.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test))

# Evaluate the model
test_loss_gru, test_accuracy_gru = model_gru.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (GRU): {test_accuracy_gru * 100:.2f}%')


**Bidirectional LSTM layers**

Bidirectional LSTM layers, which process sequences bidirectionally, improving temporal context understanding. The model is compiled with the Adam optimizer and binary crossentropy loss, is suitable for tasks like sentiment analysis or stress detection due to its proficiency in understanding contextual nuances.

In [ ]:

# Create a Sequential model with Bidirectional LSTM
model_bidirectional_lstm = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    Bidirectional(LSTM(lstm_units)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Assuming binary classification
])

# Compile the model
model_bidirectional_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 10
batch_size = 32
model_bidirectional_lstm.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test))

# Evaluate the model
test_loss_bidirectional_lstm, test_accuracy_bidirectional_lstm = model_bidirectional_lstm.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (Bidirectional LSTM): {test_accuracy_bidirectional_lstm * 100:.2f}%')

NN

  **Embedding Layer:** Converts word index sequences into dense vectors of fixed size.

  **GlobalAveragePooling1D():** Performs pooling operation to aggregate information from embeddings.
  
  **Dense Layers with Dropout:** Dense (fully connected) layers with ReLU activation functions and Dropout layers to prevent overfitting.

In [ ]:

# Create a Sequential model with a more complex Neural Network
model_nn = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length),
    GlobalAveragePooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(64, activation='relu'),
    Dropout(0.3),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Assuming binary classification
])
# accuracy as a metric to evaluate model performance.


Uses the Adam optimizer and binary crossentropy loss, suitable for binary classification problems

In [ ]:

# Compile the model with the Adam optimizer and binary crossentropy loss
model_nn.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Introduce early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
epochs = 50
batch_size = 64
history_nn = model_nn.fit(X_train_padded, y_train, epochs=epochs, batch_size=batch_size,
                           validation_data=(X_test_padded, y_test), callbacks=[early_stopping])

# Evaluate the model
test_loss_nn, test_accuracy_nn = model_nn.evaluate(X_test_padded, y_test)
print(f'Test Accuracy (Neural Network): {test_accuracy_nn * 100:.2f}%')


Best score ? Best prediction ?

In [ ]:
# Plot training history (optional)
plt.plot(history_nn.history['accuracy'], label='Training Accuracy')
plt.plot(history_nn.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.legend()
plt.show()


Y-Axis (Accuracy): Represents the accuracy of the model on the training data.

X-Axis (Epoch): Indicates the number of training epochs.

# Test part

Same example as Part2, lets test

In [ ]:
# Les textes à classifier
text1 = """Work today was tough. Dealing with unexpected challenges and tight deadlines.
The pressure is on, and it's affecting my performance."""

text2 = """Had a challenging day at work, but I managed to handle it well.
Received positive feedback, boosting my confidence."""

text3 = """The workload is piling up, and the constant pressure from the boss is overwhelming.
I'm not sure how much longer I can handle this."""

text4 = """Finished my tasks early today. Enjoying a sense of accomplishment and looking forward to a relaxing evening."""

text5 = """
A:Hey, Sarah. The brainstorming session today was intense, huh?

B:Yeah, Mark. Intense is one word for it. It feels like the pressure is on to deliver something groundbreaking.

A:Exactly. The expectations are high, and we need to step up. There's a lot riding on this project.

A:I can't shake off this feeling of unease. What if we miss something important? The stakes seem so high.

B:I get it, Sarah. But pressure can sometimes bring out the best in us. Let's channel that stress into motivation.

A:I hope so, Mark. I just worry that we're juggling too much. The workload is starting to feel overwhelming.

B:It's natural to feel that way. Let's take it one step at a time and lean on each other for support. We got this!
"""

# Tokeniser et traiter les textes
new_sequences = tokenizer.texts_to_sequences([text1, text2, text3, text4, text5])
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Faire des prédictions pour chaque modèle
predictions_lstm = model_lstm.predict(new_padded_sequences)
predictions_cnn = model_cnn.predict(new_padded_sequences)
predictions_gru = model_gru.predict(new_padded_sequences)
predictions_nn = model_nn.predict(new_padded_sequences)

# Convertir les prédictions en étiquettes binaires
binary_predictions_lstm = [1 if pred > 0.5 else 0 for pred in predictions_lstm]
binary_predictions_cnn = [1 if pred > 0.5 else 0 for pred in predictions_cnn]
binary_predictions_gru = [1 if pred > 0.5 else 0 for pred in predictions_gru]
binary_predictions_nn = [1 if pred > 0.5 else 0 for pred in predictions_nn]

# Afficher les résultats
print("Résultats pour le modèle LSTM : ", binary_predictions_lstm)
print("Résultats pour le modèle CNN : ", binary_predictions_cnn)
print("Résultats pour le modèle GRU : ", binary_predictions_gru)
print("Résultats pour le modèle NN : ", binary_predictions_nn)


In [ ]:
# Install the required libraries
!pip install transformers


In [ ]:

# Import the necessary libraries

from transformers import BertTokenizer, TFBertModel


# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and pad sequences for BERT
X_bert = df1['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=128, padding='max_length', truncation=True))
X_bert = np.array(list(X_bert))

# Split the data
X_train_bert, X_test_bert, y_train_bert, y_test_bert = train_test_split(X_bert, df1['label'], test_size=0.2, stratify=df1['label'], random_state=0)

# BERT Model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Define the BERT-based model using functional API
input_layer = Input(shape=(128,), dtype='int32')
bert_output = bert_model(input_layer)[0]
pooled_output = GlobalMaxPooling1D()(bert_output)
output_layer = Dense(1, activation='sigmoid')(pooled_output)

model_bert = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model_bert.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 1
batch_size = 12
model_bert.fit(X_train_bert, y_train_bert, epochs=epochs, batch_size=batch_size, validation_data=(X_test_bert, y_test_bert))

# Evaluate the model
test_loss_bert, test_accuracy_bert = model_bert.evaluate(X_test_bert, y_test_bert)
print(f'Test Accuracy (BERT): {test_accuracy_bert * 100:.2f}%')

# Make predictions for the BERT model
predictions_bert = model_bert.predict(X_test_bert)
binary_predictions_bert = [1 if pred > 0.5 else 0 for pred in predictions_bert]

# Display the results
print("Résultats pour le modèle BERT : ", binary_predictions_bert)


# RESUME

In [ ]:
import matplotlib.pyplot as plt

# Accuracy values from the trained models
accuracies = [test_accuracy_lstm, test_accuracy_cnn, test_accuracy_gru, test_accuracy_nn, test_accuracy_bidirectional_lstm]
models = ['LSTM', 'CNN', 'GRU', 'NN', 'Bidirectional LSTM']

# Create a bar graph
plt.bar(models, accuracies, color=['blue', 'orange', 'green', 'red', 'purple'])
plt.xlabel('Model')
plt.ylabel('Test Accuracy')
plt.title('Comparison of Model Accuracies')
plt.ylim(0, 1)  # Set the y-axis limit to the range of accuracy (0 to 1)

# Display the values on top of the bars
for i, value in enumerate(accuracies):
    plt.text(i, value + 0.01, f'{value:.2f}', ha='center', va='bottom')

# Show the plot
plt.show()


# Conclusion :

Neural Network (NN) emerges as the preeminent model for stress recognition, showcasing its remarkable capabilities in understanding intricate patterns and nuances associated with human emotions. However, it is essential to acknowledge the inherent challenges that persist in accurately identifying stress through machine learning algorithms. 

Stress, being a highly subjective and complex emotional state, presents difficulties for machines to discern consistently. Integrating additional features, such as personal or medical data, could potentially elevate the precision and efficacy of stress detection models. As we continue to advance in the field of artificial intelligence, refining and expanding these models with diverse datasets and incorporating nuanced features will be crucial for enhancing their performance in recognizing the subtle manifestations of stress in individuals.

Nicolas WATTENHOFFER